<a href="https://colab.research.google.com/github/srujaan/DL-with-PyTorch/blob/master/find_units.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import torch

In [21]:
# "t_c" is temperature in celcius
# "t_u" is temperature are unknown units
t_c = [0.5, 14.0, 15.0, 28.0, 11.0, 8.0, 3.0, -4.0, 6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)

In [22]:
# model
def model(t_u, w, b):
  return w * t_u + b

In [23]:
# loss function
def loss_fn(t_p, t_c):
  squared_diffs = (t_p - t_c) ** 2
  return squared_diffs.mean()

In [24]:
# initialize w,b parameteres
w = torch.ones(())
b = torch.ones(())

In [25]:
t_p = model(t_u, w, b)
t_p

tensor([36.7000, 56.9000, 59.2000, 82.9000, 57.3000, 49.9000, 34.9000, 22.8000,
        49.4000, 61.4000, 69.4000])

In [26]:
# check value of loss
loss = loss_fn(t_p, t_c)
loss

tensor(1847.4844)

In [27]:
# Decreasing loss

delta = 0.1

loss_rate_of_change_w = \
  (loss_fn(model(t_u, w + delta, b), t_c) - 
   loss_fn(model(t_u, w - delta, b), t_c)) / (2.0 * delta)

In [28]:
# learning rate should be low as possible

learning_rate = 1e-2
w = w - learning_rate * loss_rate_of_change_w

In [29]:
# we can do same for b

loss_rate_of_change_b = \
  (loss_fn(model(t_u, w, b + delta), t_c) - 
   loss_fn(model(t_u, w, b - delta), t_c)) / (2.0 * delta)

b = b - learning_rate * loss_rate_of_change_b

In [30]:
def dloss_fn(t_p, t_c):
  dsq_diffs = 2 * (t_p - t_c) / t_p.size(0) # The division is from the derivative mean
  return dsq_diffs


In [31]:
# Applying derivatives to the model

def dmodel_dw(t_u, w, b):
  return t_u

def dmodel_db(t_u, w, b):
  return 1.0

In [32]:
# Putting all this together

def grad_fn(t_u, t_c, t_p, w, b):
  dloss_dtp = dloss_fn(t_p, t_c)
  dloss_dw = dloss_dtp * dmodel_dw(t_u, w, b)
  dloss_db = dloss_dtp * dmodel_db(t_u, w, b)
  return torch.stack([dloss_dw.sum(), dloss_db.sum()])

In [50]:
# Training loop

def training_loop(n_epochs, learning_rate, params, t_u, t_c, print_params=True):
  for epoch in range(1, n_epochs + 1):
    w, b = params

    t_p = model(t_u, w, b)  # Forward pass
    loss = loss_fn(t_p, t_c)
    grad = grad_fn(t_u, t_c, t_p, w, b) # Backward pass

    params = params - learning_rate * grad
    if epoch in {1, 2, 3, 10, 99, 100, 4000, 5000}:
      print('Epoch %d, Loss %f' % (epoch, float(loss)))
      if print_params:
        print('  Params: ', params)
        print('  Grad:', grad)

    if epoch in {4, 12, 101}:
      print('...')

    if not torch.isfinite(loss).all():
      break

  return params


In [51]:
training_loop(
    n_epochs = 100,
    learning_rate = 1e-2,
    params = torch.tensor([1.0, 0.0]),
    t_u = t_u,
    t_c = t_c)

Epoch 1, Loss 1763.884766
  Params:  tensor([-44.1730,  -0.8260])
  Grad: tensor([4517.2964,   82.6000])
Epoch 2, Loss 5802484.500000
  Params:  tensor([2568.4011,   45.1637])
  Grad: tensor([-261257.4062,   -4598.9702])
Epoch 3, Loss 19408029696.000000
  Params:  tensor([-148527.7344,   -2616.3931])
  Grad: tensor([15109614.0000,   266155.6875])
...
Epoch 10, Loss 90901105189019073810297959556841472.000000
  Params:  tensor([3.2144e+17, 5.6621e+15])
  Grad: tensor([-3.2700e+19, -5.7600e+17])


tensor([-1.8590e+19, -3.2746e+17])

In [52]:
# Decrease the learning rate and try again...
# these are called hyperparameters

training_loop(
    n_epochs = 100,
    learning_rate = 1e-4,
    params = torch.tensor([1.0, 0.0]),
    t_u = t_u,
    t_c = t_c)

Epoch 1, Loss 1763.884766
  Params:  tensor([ 0.5483, -0.0083])
  Grad: tensor([4517.2964,   82.6000])
Epoch 2, Loss 323.090515
  Params:  tensor([ 0.3623, -0.0118])
  Grad: tensor([1859.5493,   35.7843])
Epoch 3, Loss 78.929634
  Params:  tensor([ 0.2858, -0.0135])
  Grad: tensor([765.4666,  16.5122])
...
Epoch 10, Loss 29.105247
  Params:  tensor([ 0.2324, -0.0166])
  Grad: tensor([1.4803, 3.0544])
...
Epoch 99, Loss 29.023582
  Params:  tensor([ 0.2327, -0.0435])
  Grad: tensor([-0.0533,  3.0226])
Epoch 100, Loss 29.022667
  Params:  tensor([ 0.2327, -0.0438])
  Grad: tensor([-0.0532,  3.0226])


tensor([ 0.2327, -0.0438])

In [53]:
# Normalize the inputs, so they are in range -1.0 to 1.0

t_un = 0.1 * t_u

# updating it in the training loop
training_loop(
    n_epochs = 100,
    learning_rate = 1e-2, # it is set high and didn't blow up
    params = torch.tensor([1.0, 0.0]),
    t_u = t_un, # updated t_u to our new, rescaled t_un.
    t_c = t_c)

Epoch 1, Loss 80.364342
  Params:  tensor([1.7761, 0.1064])
  Grad: tensor([-77.6140, -10.6400])
Epoch 2, Loss 37.574913
  Params:  tensor([2.0848, 0.1303])
  Grad: tensor([-30.8623,  -2.3864])
Epoch 3, Loss 30.871077
  Params:  tensor([2.2094, 0.1217])
  Grad: tensor([-12.4631,   0.8587])
...
Epoch 10, Loss 29.030489
  Params:  tensor([ 2.3232, -0.0710])
  Grad: tensor([-0.5355,  2.9295])
...
Epoch 99, Loss 22.214186
  Params:  tensor([ 2.7508, -2.4910])
  Grad: tensor([-0.4453,  2.5208])
Epoch 100, Loss 22.148710
  Params:  tensor([ 2.7553, -2.5162])
  Grad: tensor([-0.4446,  2.5165])


tensor([ 2.7553, -2.5162])

In [54]:
# see if the params change for n_epochs = 5000

params = training_loop(
    n_epochs = 5000,
    learning_rate = 1e-2,
    params = torch.tensor([1.0, 0.0]),
    t_u = t_un,
    t_c = t_c,
    print_params = False
  )
params

Epoch 1, Loss 80.364342
Epoch 2, Loss 37.574913
Epoch 3, Loss 30.871077
...
Epoch 10, Loss 29.030489
...
Epoch 99, Loss 22.214186
Epoch 100, Loss 22.148710
...
Epoch 4000, Loss 2.927680
Epoch 5000, Loss 2.927648


tensor([  5.3671, -17.3012])